In [1]:
import open3d as o3d
import numpy as np


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
mesh = o3d.io.read_triangle_mesh("assets/data/ios_with_smilearch.stl")
mesh.compute_vertex_normals()


TriangleMesh with 492809 points and 164284 triangles.

In [3]:
mesh

TriangleMesh with 492809 points and 164284 triangles.

In [8]:
def create_coordinate_arrows(size=1.0, origin=[0, 0, 0]):
    """
    좌표축을 화살표로 생성하는 함수
    
    Args:
        size: float - 화살표 크기
        origin: list - 좌표축 원점 위치 [x, y, z]
    
    Returns:
        list: 화살표 geometry 리스트
    """
    geometries = []
    
    # 화살표 생성을 위한 실린더와 원뿔 생성
    for i, color in enumerate([[1,0,0], [0,1,0], [0,0,1]]):  # RGB for XYZ
        # 실린더 (화살표 몸통) 생성
        cylinder = o3d.geometry.TriangleMesh.create_cylinder(
            radius=size/30,
            height=size*0.8
        )
        # 실린더를 중심이 원점에 오도록 이동
        cylinder.translate([0, 0, size*0.4])
        
        # 원뿔 (화살표 머리) 생성
        cone = o3d.geometry.TriangleMesh.create_cone(
            radius=size/15,
            height=size*0.2
        )
        # 원뿔을 실린더 위에 배치
        cone.translate([0, 0, size*0.8])
        
        # 화살표 합치기
        arrow = cylinder + cone
        
        # 각 축 방향으로 회전
        if i == 0:  # X축
            arrow.rotate(
                o3d.geometry.get_rotation_matrix_from_xyz([0, np.pi/2, 0])
            )
        elif i == 1:  # Y축
            arrow.rotate(
                o3d.geometry.get_rotation_matrix_from_xyz([-np.pi/2, 0, 0])
            )
        
        # 원점으로 이동
        arrow.translate(origin)
        
        # 색상 지정
        arrow.paint_uniform_color(color)
        geometries.append(arrow)
    
    return geometries


In [10]:
# 메쉬 크기에 맞춰 좌표축 크기 설정
mesh_bbox = mesh.get_axis_aligned_bounding_box()
mesh_size = mesh_bbox.get_extent()
axis_size = max(mesh_size) * 0.2

# 좌표축 화살표 생성
coordinate_arrows = create_coordinate_arrows(size=axis_size)

# 메쉬와 좌표축 함께 시각화
geometries = [mesh] + coordinate_arrows
o3d.visualization.draw_geometries(geometries)

## 여기까지 모델 + 화살표 그리기

In [12]:
def show(mesh):
    mesh_bbox = mesh.get_axis_aligned_bounding_box()
    mesh_size = mesh_bbox.get_extent()
    axis_size = max(mesh_size) * 0.2

    # 좌표축 화살표 생성
    coordinate_arrows = create_coordinate_arrows(size=axis_size)

    # 메쉬와 좌표축 함께 시각화
    geometries = [mesh] + coordinate_arrows
    o3d.visualization.draw_geometries(geometries)

In [15]:
random_angles = [np.random.random()*np.pi*2 for _ in range(3)]
mesh.rotate(o3d.geometry.get_rotation_matrix_from_xyz(random_angles), center=mesh.get_center())

TriangleMesh with 492809 points and 164284 triangles.

In [16]:
show(mesh)

### 자 이제 정렬을 해보자
- 전체 법선 벡터를 가지고 